In [27]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import BpeTrainer

In [5]:
# convert the raw data with labels to just tokens
fh = open("data/uk_openaddresses_formatted_addresses_tagged.random.tsv", "r")
fh_out = open("data/uk_openaddresses_formatted_addresses_raw.txt", "w")

for line in fh:
    line = line.strip().lower()  # lower case b/c our fec data is lower cased
    line_data = [i.split('/')[0] for i in line.split("\t")[2].split(' ') if 'fsep' not in i]
    line_data = ' '.join(line_data)
    fh_out.write(line_data + "\n")

In [51]:
tokenizer = Tokenizer(BPE())
tokenizer.add_special_tokens(["[CLS]", "[SEP]", "[MASK]", "[PAD]"])
tokenizer.pre_tokenizer = Whitespace()

In [52]:
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=(
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ),
)
tokenizer.enable_padding(length=25)

In [53]:
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=30000, min_frequence=2)
tokenizer.train(files=["data/uk_openaddresses_formatted_addresses_raw.txt"], trainer=trainer)

In [54]:
e = tokenizer.encode_batch(["rock road wadebridge united kingdom pl27 6nw"])

In [58]:
tokenizer.save("tokenizers/ukonly.json")

In [17]:
tokenizer.encode("rock road wadebridge united kingdom pl27 6nw", max_length=25)

TypeError: PyTokenizer.encode() got an unexpected keyword argument: max_length

In [55]:
e = e[0]

In [66]:
e.attention_mask

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [64]:
tokenizer.token_to_id('sw')

100